In [13]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import load_df
from tweedejaars_project import show_real_penalty_score, show_time_diff_score
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *


df = load_df()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
features = [
    "import_capacity",
    "settlement_price_bestguess",
]

train_x = df[features][:165195]
train_y = df['target_two_sided_ptu'][:165195]

test_x = df[features][165195:]
test_y = df['target_two_sided_ptu'][165195:]

# Initialize the Decision Tree Classifier
clf = HistGradientBoostingClassifier()

# Train the classifier
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

print("Classification Report:")
print(classification_report(test_y, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_y, y_pred))

0.935187371709869
Classification Report:
              precision    recall  f1-score   support

       False       0.94      1.00      0.97     51710
        True       0.08      0.01      0.01      3341

    accuracy                           0.94     55051
   macro avg       0.51      0.50      0.49     55051
weighted avg       0.89      0.94      0.91     55051

Confusion Matrix:
[[51462   248]
 [ 3320    21]]


In [18]:
show_real_penalty_score(df, df['target_two_sided_ptu'], df['target_two_sided_ptu_realtime'], df['ptu_id'])

False negative score (pred/max): 1.0, 3737/3737
False positive score (pred/max): 1.0, 0/21548


In [16]:
show_time_diff_score(df, df['target_two_sided_ptu'], df['ptu_id'])

               neg  pos
count  1454.000000  2.0
mean     -9.472490  0.0
std       2.981752  0.0
min     -14.000000  0.0
25%     -12.000000  0.0
50%      -9.000000  0.0
75%      -7.000000  0.0
max      -3.000000  0.0
Time taken (pred/max): -9.459478021978022/-9.460851648351648, using 1456/1456


In [17]:
false_neg_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, np.NaN, 0, -100, -150, 
        -200, -300, -100, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, 20, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        40, 50, 90, 200, 80, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        40, 50, 90, 200, 200, 
        200, 200, 200
    ],
    'target_two_sided_ptu': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, False, True, True, True, True, True, False, False,
        False, False, False, False, False, False,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_pos_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, -100, 0, -100, -150, 
        -200, -300, -100, -100, -100, 
        -100, -100, -100, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        -300, -300, -300, -300, -300, 
        -300, -300, -300
    ],
    'target_two_sided_ptu': [
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_neg_test_df = pd.DataFrame(false_neg_test_df)
false_pos_test_df = pd.DataFrame(false_pos_test_df)

false_neg = show_real_penalty_score(false_neg_test_df, false_neg_test_df['target_two_sided_ptu'], false_neg_test_df['pred'], false_neg_test_df['ptu_id'])
false_pos = show_real_penalty_score(false_pos_test_df, false_pos_test_df['target_two_sided_ptu'], false_pos_test_df['pred'], false_pos_test_df['ptu_id'])
print(false_neg)
print(false_pos)

False negative score (pred/max): 0.4, 2/5
False positive score (pred/max): 0.4, 0/0
False negative score (pred/max): nan, 0/0
False positive score (pred/max): nan, 3/14
None
None


/home/nub/tweedejaars_project/tweedejaars_project/evaluation/metrics.py:80: RuntimeWarning: invalid value encountered in scalar divide
  # TODO fix it?
/home/nub/tweedejaars_project/tweedejaars_project/evaluation/metrics.py:81: RuntimeWarning: invalid value encountered in scalar divide
  def compute_time_diff_flip(df: pd.DataFrame, pred: pd.Series, ids: pd.Series):
